# Out-of-Sample Equity Premium Prediction: Combination Forecasts and Links to the Real Economy

Source: https://doi.org/10.1093/rfs/hhp063

## 1. Introduction

The study preprocesses market and macroeconomic data, constructs a set of predictive variables, and generates out-of-sample equity premium forecasts using rolling regression models. 

It implements both single-factor and multi-factor combination forecasting methods, including simple (mean, median, trimmed mean) and weighted (DMSPE) approaches.

The predictive performance of each model is evaluated using metrics such as mean squared error (MSE), out-of-sample R², statistical significance (p-values), and economic returns. 

Visualization routines are included to illustrate time-series prediction error and comparative model effectiveness.

The results show that ensemble forecast combinations consistently outperform single-factor predictors in terms of accuracy and stability.

In [ ]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

## 2. Preprocessing and Feature Engineering

Domestic data is missing for B/M (Book-to-Market Ratio), company bond yield-related variables such as DFY and DFR, and macroeconomic variables such as INFL and I/K. Perform combined forecasting using the remaining ten variables.

In [ ]:
# Construction of price-related variables for the CSI 300 index: D/P, D/Y, P/E, SVAR
pedata = pd.read_excel("000300.SH-历史PE_PB-20241202.xlsx", header=0)
pedata["lastclsprc"] = pedata["收盘价"].shift(1)
pedata["交易日期"] = pd.to_datetime(pedata["交易日期"],format="%Y-%m-%d")
pedata["Year"] = pedata["交易日期"].dt.year
pedata["Month"] = pedata["交易日期"].dt.month
pedata.loc[pedata["Month"]<=3,"Season"] =1
pedata.loc[(pedata["Month"]>=3) & (pedata["Month"]<=6),"Season"] =2
pedata.loc[(pedata["Month"]>=6) & (pedata["Month"]<=9),"Season"] =3
pedata.loc[(pedata["Month"]>=9) & (pedata["Month"]<=12),"Season"] =4
pedata["Dividend"] = pedata["收盘价"] * pedata["股息率"]/100
pedata["D/P"] = np.log(pedata["股息率"])

data = pedata.groupby(["Year","Season"]).apply(lambda x: x["D/P"].mean()).reset_index()
pedata["D/Y"] = np.log(pedata["Dividend"]/pedata["lastclsprc"])
data1 = pedata.groupby(["Year","Season"]).apply(lambda x: x["D/Y"].mean()).reset_index()
data = data.merge(data1, on=["Year","Season"], how="inner")
pedata["E/P"] = -np.log(pedata["市盈率TTM"])
data1 = pedata.groupby(["Year","Season"]).apply(lambda x: x["E/P"].mean()).reset_index()
data = data.merge(data1, on=["Year","Season"], how="inner")
pedata["SVAR"] = ((pedata["收盘价"] - pedata["lastclsprc"])/pedata["lastclsprc"])**2
data1 = pedata.groupby(["Year","Season"]).apply(lambda x: x["SVAR"].sum()).reset_index()
data = data.merge(data1, on=["Year","Season"], how="outer",suffixes=('', '_y'))
data.columns = ["Year","Season","D/P","D/Y","P/E","SVAR"]
data.drop([len(data)-1],inplace=True)
data.dropna(inplace=True)
data.head(5)

In [ ]:
# Construction of the CSI 300 index dividend payout ratio variable (D/E)
statementdata = pd.read_excel("000300.SH-财务数据升-20241202.xlsx").T
statementdata.reset_index(inplace=True)
statementdata.iloc[0,1] ="Season"
statementdata.columns = statementdata.iloc[0,:]
statementdata.drop([0],inplace=True)

k=1
for i in statementdata["Season"].unique():
    statementdata.loc[statementdata["Season"]==i,"Season"] = k
    k+=1
statementdata["Year"] = pd.to_datetime(statementdata["报告期"], format="%Y-%m-%d").dt.year

data1 = pedata.groupby(["Year","Season"]).apply(lambda x: x["Dividend"].mean()).reset_index()
data1.drop([0,1],inplace=True)
data1 = data1.merge(statementdata[["Year","Season","归属母公司股东的净利润"]], on=["Year","Season"], how="outer")
data1.dropna(inplace=True)
data1.columns = ["Year","Season","Dividend","NetProfit"]
data1["D/E"] = np.log(data1["Dividend"]/data1["NetProfit"].apply(lambda x: x.replace(",","")).astype(float))
data = data.merge(data1[["Year","Season","D/E"]], on=["Year","Season"], how="outer")


In [ ]:
# Construction of the net equity expansion variable (NTIS) for Shanghai A-shares
Annualcap = pd.read_csv("TRD_Yearm.csv")
Annualcap = Annualcap[(Annualcap["Markettype"]==1) | (Annualcap["Markettype"]==32)].groupby("Trdynt").apply(lambda x: x["Ymvttl"].sum()).reset_index()
Annualcap.index = range(len(Annualcap))
Annualcap.columns = ["Year","AnnualCap"]
Annualcap["Season"] = 4

issueSH = pd.read_csv("IPO_Ipobasic.csv")
issueSH = issueSH[(issueSH["Stkcd"]<700000) & (issueSH["Stkcd"]>=600000) & (issueSH["T1"]=="A")]
issueSH = issueSH.sort_values(by="Listdt").groupby("Listdt").sum().reset_index()
issueSH.drop(issueSH[issueSH["Nshripo"]==0].index, axis =0,inplace=True)
issueSH["Year"] = pd.to_datetime(issueSH["Listdt"], format="%Y-%m-%d").dt.year
issueSH["Month"] = pd.to_datetime(issueSH["Listdt"], format="%Y-%m-%d").dt.month
issueSH.loc[issueSH["Month"]<=3,"Season"] =1
issueSH.loc[(issueSH["Month"]>=3) & (issueSH["Month"]<=6),"Season"] =2
issueSH.loc[(issueSH["Month"]>=6) & (issueSH["Month"]<=9),"Season"] =3
issueSH.loc[(issueSH["Month"]>=9) & (issueSH["Month"]<=12),"Season"] =4
issueSH = issueSH.groupby(["Year","Season"]).apply(lambda x: x["Nshripo"].sum()).reset_index()
issueSH.columns = ["Year","Season","Nshripo"]
issue14 = issueSH.iloc[0:5,:]
issueSH = data[["Year","Season"]].merge(issueSH, on=["Year","Season"], how="outer")
issueSH.fillna(0,inplace=True)
issueSH.sort_values(by=["Year","Season"],inplace=True)

issueSH["issueTTM"] = issueSH["Nshripo"].rolling(4).sum()
issueSH = issueSH.merge(Annualcap, on=["Year","Season"], how="outer")
issueSH.ffill(inplace=True)
issueSH.drop(issueSH.iloc[0:5,:].index, axis=0,inplace=True)
issueSH["NTIS"] = issueSH["issueTTM"]/issueSH["AnnualCap"]*100

data1 = issueSH[["Year","Season","NTIS"]]
data = data.merge(data1, on=["Year","Season"], how="inner")

In [ ]:
# Construction of the 3-month treasury bill rate variable (using yield as a replacement)
ytm = pd.read_csv("BND_TreasYield.csv")
ytm025 = ytm[(ytm["Cvtype"]==1)&(ytm["Yeartomatu"]==0.25)]
ytm025["Year"] = pd.to_datetime(ytm025["Trddt"], format="%Y-%m-%d").dt.year
ytm025["Month"] = pd.to_datetime(ytm025["Trddt"], format="%Y-%m-%d").dt.month
ytm025.loc[ytm025["Month"]<=3, "Season"] = 1
ytm025.loc[(ytm025["Month"]>=3) & (ytm025["Month"]<=6), "Season"] = 2
ytm025.loc[(ytm025["Month"]>=6) & (ytm025["Month"]<=9), "Season"] = 3
ytm025.loc[(ytm025["Month"]>=9) & (ytm025["Month"]<=12), "Season"] = 4

data1 = ytm025.groupby(["Year","Season"]).apply(lambda x: x["Yield"].mean()).reset_index()
data1.rename({0:"TBL"},axis=1,inplace=True)
data = data.merge(data1, on=["Year","Season"], how="inner")

In [ ]:
# Construction of the long-term government bond yield variable (LTY)
ytm = ytm[(ytm["Cvtype"]==1)&(ytm["Yeartomatu"]==10)]
ytm["Year"] = pd.to_datetime(ytm["Trddt"], format="%Y-%m-%d").dt.year
ytm["Month"] = pd.to_datetime(ytm["Trddt"], format="%Y-%m-%d").dt.month
ytm.loc[ytm["Month"]<=3, "Season"] = 1
ytm.loc[(ytm["Month"]>=3) & (ytm["Month"]<=6), "Season"] = 2
ytm.loc[(ytm["Month"]>=6) & (ytm["Month"]<=9), "Season"] = 3
ytm.loc[(ytm["Month"]>=9) & (ytm["Month"]<=12), "Season"] = 4

data1 = ytm.groupby(["Year","Season"]).apply(lambda x: x["Yield"].mean()).reset_index()
data1.rename({0:"LTY"},axis=1,inplace=True)
data = data.merge(data1, on=["Year","Season"], how="inner")

In [ ]:
# Construction of the long-term government bond return variable (LTR)
Binfo = pd.read_csv("BND_Florate.csv")
interrate = Binfo[(Binfo["Term"]==10)]

interrate["Year"] = pd.to_datetime(interrate["Stopdt"], format="%Y-%m-%d").dt.year
interrate["Month"] = pd.to_datetime(interrate["Stopdt"], format="%Y-%m-%d").dt.month
interrate.loc[interrate["Month"]<=3, "Season"] = 1
interrate.loc[(interrate["Month"]>=3) & (interrate["Month"]<=6), "Season"] = 2
interrate.loc[(interrate["Month"]>=6) & (interrate["Month"]<=9), "Season"] = 3
interrate.loc[(interrate["Month"]>=9) & (interrate["Month"]<=12), "Season"] = 4

data1 = interrate.groupby(["Year","Season"]).apply(lambda x: x["Intrrate"].mean()).reset_index()
data1.rename({0:"LTR"},axis=1,inplace=True)
data = data.merge(data1, on=["Year","Season"], how="outer")
data.drop(data.iloc[-1:,:].index,axis=0,inplace=True)
data.ffill(inplace=True) #Forward fill missing values

In [ ]:
# Construction of the term spread variable (TMS, difference between long-term and short-term Treasury yields)
data["TMS"] = data["LTY"] - data["TBL"]

In [ ]:
# Risk Premium
rpremium = pd.read_csv("STK_MKT_THRFACMONTH.csv")
rpremium = rpremium[rpremium["MarkettypeID"]=="P9701"]
rpremium[["Year","Month"]] = rpremium["TradingMonth"].str.split("-",expand=True)
rpremium["Month"] = rpremium["Month"].astype(int)
rpremium["Year"] = rpremium["Year"].astype(int)
rpremium.loc[rpremium["Month"]<=3, "Season"] = 1
rpremium.loc[(rpremium["Month"]>3) & (rpremium["Month"]<=6), "Season"] = 2
rpremium.loc[(rpremium["Month"]>6) & (rpremium["Month"]<=9), "Season"] = 3
rpremium.loc[(rpremium["Month"]>9) & (rpremium["Month"]<=12), "Season"] = 4
data1 = rpremium.groupby(["Year", "Season"]).apply(lambda x: x["RiskPremium2"].sum()).reset_index()
data1.rename({0:"RiskPremium"},axis=1,inplace=True)
data = data.merge(data1, on=["Year","Season"], how="inner")
data.head(5)

## 3. Univariate Forecast Model Training and Regression

Perform regression and forecasting on one-period lagged excess returns; 'predict' contains forecasts from each individual variable. **Compared with historical mean of the return as baseline model**.

In [ ]:
train = 25 #1/3 of data
values = pd.DataFrame()
predict = data.loc[train:,["Year","Season","RiskPremium"]].reset_index(drop=True)
for k in range(len(data)-train):
    predict.loc[k,"r_Histmean"] = data.loc[1:train+k-1,"RiskPremium"].mean()
data.columns[2:-1]
for i in data.columns[2:-1]:
    values[i] = sm.OLS(data.loc[:train,"RiskPremium"],sm.add_constant(data.loc[:train,i])).fit().params.values
    predict[i] = values.loc[0,i]+values.loc[1,i]*data.loc[train-1:len(data)-1,i].reset_index(drop=True)
predict.head(5)

## 4. Combined Model and  Forecast Construction 

Combined Model contains two types of combining method. 

First type includes mean, median and trimmed mean of ten singular variate.  

Second type includes DMSPE prediction.

In [ ]:
# Forecasts from the first type of combined forecasting
values.index = ["alpha","beta"]
combpredict = predict[["Year","Season"]]
combpredict["r_Histmean"] = predict["r_Histmean"]
combpredict["RiskPremium"] = data.loc[train:,"RiskPremium"].reset_index(drop=True)
combpredict["Mean"] = predict.iloc[:,2:].mean(axis=1)
combpredict["Median"] = predict.iloc[:,2:].median(axis=1)
combpredict["TrimmedMean"] = (predict.iloc[:,2:].sum(axis=1)-predict.iloc[:,2:].max(axis=1)-predict.iloc[:,2:].min(axis=1))/(predict.iloc[:,2:].shape[1]-2)
combpredict_1 = combpredict.copy()
combpredict_1.head()

In [ ]:
# Forecasts from the second type of combined forecasting
# Theta is 1 or 0.9，q0 is holdout period of out-of-sample period
def DMSPE_prediction(theta,q0):
    fai = pd.DataFrame()
    for j in range(len(predict)-q0):
        for i in predict.columns[4:]:
            fai.loc[j,i] = (np.full(q0, theta) ** range(q0-1,-1,-1)) @ ((predict.loc[j:j+q0-1,"RiskPremium"]-predict.loc[j:j+q0-1,i])**2).values.T
    weights = (1/fai).div((1/fai).sum(axis=1),axis=0)
    new_predict = (weights.values * predict.iloc[q0:,4:].values).sum(axis=1)
    return new_predict

q0 = 15 # manually setting holdout period
combpredict = combpredict.iloc[q0:,:].reset_index(drop=True)
combpredict["DMSPE_0.9"] = DMSPE_prediction(0.9,q0)
combpredict["DMSPE_1"] = DMSPE_prediction(1,q0)
combpredict.head()

## 5. Results Plotting and Visualization

Generate the mean squared error (MSE) between historical average predictions and actual returns, and the difference with the MSE between individual variable predictions and actual returns. Cumulative error plot is shown in the results.

In [ ]:
plt.figure(figsize=(15,8))
j=1
for i in predict.columns[4:]:
    serie = ((predict["RiskPremium"]-predict["r_Histmean"])**2)-((predict["RiskPremium"]-predict[i])**2)
    cumsum_serie = np.cumsum(serie)
    plt.subplot(3,4,j)
    plt.subplots_adjust(hspace=0.5,wspace=0.3)
    plt.plot(cumsum_serie)
    plt.title(i)
    plt.axhline(0, color='black', lw=1)
    plt.ylim(-0.1,0.1)
    j+=1
plt.show()

Generate the MSE between historical average predictions and actual returns, and the difference with the MSE between combined forecasts and actual returns. Cumulative error plot is shown in the results.

In [ ]:
plt.figure(figsize=(12,5))
j=1
for i in combpredict.columns[4:]:
    serie = ((combpredict["RiskPremium"]-combpredict["r_Histmean"])**2)-((combpredict["RiskPremium"]-combpredict[i])**2)
    cumsum_serie = np.cumsum(serie)
    plt.subplot(2,3,j)
    plt.subplots_adjust(hspace=0.5,wspace=0.3)
    plt.plot(cumsum_serie)
    plt.title(i)
    plt.axhline(0, color='black', lw=1)
    plt.ylim(-0.1,0.1)
    j+=1
plt.show()

Findings: It is shown that only P/E, D/E, LTR has a relative low cumulative error among univariate models. 

Comparing to univariate models, combined models predict cumulative error on a greater stability, while the variation of the error of second type of combination is much more robust.

## 6. R2OS Metric and Significance Testing

R²OS (Out-of-sample R-squared) measures a model’s predictive performance out of sample; higher positive values indicate better out-of-sample explanatory power.

In [ ]:
# R^2OS values for individual variable and combined forecasts
r2os = pd.DataFrame()
for i in predict.columns[4:]:
    r2os.loc[i,"R2OS"] = 1 - np.sum((predict[i]-predict["RiskPremium"])**2)/np.sum((predict["RiskPremium"]-predict["r_Histmean"])**2)
for i in combpredict.columns[4:]:
    r2os.loc[i,"R2OS"] = 1 - np.sum((combpredict[i]-combpredict["RiskPremium"])**2)/np.sum((combpredict["RiskPremium"]-combpredict["r_Histmean"])**2)

In [ ]:
# p-values from the adjusted MSPE test of R^2OS for individual variable and combined forecasts.
for i in predict.columns[4:]:
    ft = (predict["RiskPremium"]-predict["r_Histmean"])**2 - (predict[i]-predict["RiskPremium"])**2 + (predict["RiskPremium"]-predict["r_Histmean"])**2
    r2os.loc[i,"p_value"] = sm.OLS(ft,np.ones(len(ft))).fit().pvalues.iloc[0]
for i in combpredict.columns[4:]:
    ft = (combpredict["RiskPremium"]-combpredict["r_Histmean"])**2 - (combpredict[i]-combpredict["RiskPremium"])**2 + (combpredict["RiskPremium"]-combpredict["r_Histmean"])**2
    r2os.loc[i,"p_value"] = sm.OLS(ft,np.ones(len(ft))).fit().pvalues.iloc[0]

In [ ]:
# Forecast evaluation: Mean-variance investor utility
estimate = data[["Year","Season"]].copy()
estimate["r_var"] = data["RiskPremium"].rolling(4).var().shift(1)
estimate = estimate.merge(predict, on=["Year","Season"], how="inner")
estimate = estimate.merge(combpredict, on=["Year","Season","RiskPremium","r_Histmean"], how="inner")
gamma = 3 # same as the paper setting
weight_bench = (1/gamma)*(estimate['r_Histmean']/estimate['r_var'])
return_bench = estimate['RiskPremium']*weight_bench
cer_bench = return_bench.mean() - 0.5*gamma*(return_bench.var())

for i in estimate.columns[5:]:
    weight_model = (1/gamma)*(estimate[i]/estimate['r_var'])
    return_model = estimate['RiskPremium']*weight_model
    cer_model = return_model.mean() - 0.5*gamma*(return_model.var())
    r2os.loc[i,"economic_return"] = cer_model - cer_bench
r2os

Findings:
Most single-factor predictors (e.g., D/P, D/Y, SVAR) report negative R²OS, meaning they fail to outperform the benchmark in out-of-sample prediction.

P/E and LTR show small positive R²OS, but the improvement is limited.

Combined models (Mean, Median, TrimmedMean, DMSPE) all have positive R²OS, showing significantly stronger out-of-sample predictive ability. Mean reaches 0.228929, the best among the displayed results.

Combination model p-values shows great significance, and their economic returns are positive, further confirming effectiveness.

Single factors rarely deliver robust out-of-sample results. Ensemble or aggregated methods provide substantial improvements in predictive power.

Difference in mean squared error between individual variable forecasts and combined forecasts are shown in the series graph

In [ ]:
plt.figure(figsize=(15,6))
j=1
for i in predict.columns[4:]:
    serie = ((predict["RiskPremium"]-predict["r_Histmean"])**2)-((predict["RiskPremium"]-predict[i])**2)
    plt.subplot(3,5,j)
    plt.subplots_adjust(hspace=0.5,wspace=0.3)
    plt.plot(serie)
    plt.title(i)
    plt.axhline(0, color='black', lw=1)
    plt.ylim(-0.05,0.05)
    j+=1
for i in combpredict.columns[4:]:
    serie = ((combpredict["RiskPremium"]-combpredict["r_Histmean"])**2)-((combpredict["RiskPremium"]-combpredict[i])**2)
    plt.subplot(3,5,j)
    plt.subplots_adjust(hspace=0.5,wspace=0.3)
    plt.plot(serie)
    plt.title(i)
    plt.axhline(0, color='black', lw=1)
    plt.ylim(-0.05,0.05)
    j+=1
plt.show()

Findings: Most single-factor models (e.g., D/P, D/Y, SVAR, TBL) exhibit large and volatile MSE, with occasional spikes, suggesting limited predictive precision and stability.

Ensemble methods (Mean, Median, TrimmedMean, DMSPE_0.9, DMSPE_1) generally maintain lower and more stable MSE, especially the DMSPE series, which demonstrates the most robust performance.

Single-factor forecasting is prone to higher and more unstable errors, while multi-factor ensemble models substantially reduce prediction error and improve consistency. Ensemble approaches are thus preferable for practical implementation.